# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [35]:
#instalando o selenium
!pip install selenium

In [36]:
#baixar o webdriver
#chrome - > chromedriver
#firefox - > 

In [37]:
#importando webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
navegador = webdriver.Chrome("chromedriver.exe") 
#passo 1 - pegar a cotação do dolar 
    #entrar no google
    #pesquisar cotação do dolar
    #pegar cotação
navegador.get("https://www.google.com/")#passando url de entrada 
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar") 
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(Keys.ENTER) #falta o build pat
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

#passo 2 - pegar a cotação do euro 
navegador.get("https://www.google.com/")#passando url de entrada 
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro") 
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(Keys.ENTER) #falta
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

#passo3 - pegar a cotação do ouro 
navegador.get("https://www.melhorcambio.com/ouro-hoje")#passando url de entrada 
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")

 

In [38]:
#passo 4 - importar a base de dados 
import pandas as pd
tabela = pd.read_excel("Produtos.xlsx")


In [39]:
#passo 5 - atualizar as cotações, preço de compra e preço de venda 

#atualizar cotacao 
tabela.loc[tabela['Moeda'] == 'Dolar',"Cotação"] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro',"Cotação"] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro',"Cotação"] = float(cotacao_ouro)

#preco de compra  preco original * cotacao 
tabela['Preço de Compra'] = tabela['Preço Base Original']*tabela['Cotação']

# preco de venda preco de compra * margem 
tabela['Preço de Venda'] = tabela['Preço de Compra'] *tabela['Margem']

#formatando casas decimais
tabela['Preço de Venda'] = tabela['Preço de Venda'].map("R${:.2f}".format)
tabela['Preço de Compra'] = tabela['Preço de Compra'].map("R${:.2f}".format)


In [40]:
#passo 6 - exportar o relatorio atualizado
tabela.to_excel("Produtos Novo.xlsx",index=False)
navegador.quit()